In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AirbnbDataProcessing") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://hdfs-namenode:9000") \
    .getOrCreate()

# Read the Airbnb data from HDFS
file_path = "hdfs://hdfs-namenode:9000/datasets" + "/AB_NYC_2019.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)


print(df.show())


+----+--------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|       host_name|neighbourhood_group|     neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|            John|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca.

## Calculate the total number of reviews per neighborhood

In [ ]:
from pyspark.sql.functions import col, count

# Handle null values: fill nulls with default values or drop rows with nulls
df = df.na.fill({
    'neighbourhood': 'Unknown',
    'number_of_reviews': 0
})

reviews_per_neighborhood = df.groupBy("neighbourhood").agg(count("number_of_reviews").alias("total_reviews"))
print(reviews_per_neighborhood.show())


+------------------+-------------+
|     neighbourhood|total_reviews|
+------------------+-------------+
|            Corona|           64|
|      Richmondtown|            1|
|      Prince's Bay|            4|
|       Westerleigh|            2|
|        Mill Basin|            4|
|      Civic Center|           52|
|          40.83166|            1|
|        Douglaston|            7|
|        Mount Hope|           20|
|       Marble Hill|           12|
|         Rego Park|          106|
|          40.81225|            1|
|     Dyker Heights|           12|
| Kew Gardens Hills|           26|
|      Dongan Hills|            7|
|          40.81078|            1|
|Financial District|          744|
|       Bay Terrace|            6|
|          40.83117|            1|
|           Midtown|         1541|
+------------------+-------------+
only showing top 20 rows

None


In [4]:
popular_neighborhoods = reviews_per_neighborhood.filter(col("total_reviews") > 100)
print(popular_neighborhoods.show())

+------------------+-------------+
|     neighbourhood|total_reviews|
+------------------+-------------+
|         Rego Park|          106|
|Financial District|          744|
|           Midtown|         1541|
|   Jackson Heights|          184|
|    Hell's Kitchen|         1951|
|   Windsor Terrace|          157|
| Greenwich Village|          390|
|      Clinton Hill|          571|
|Washington Heights|          895|
|  Ditmars Steinway|          306|
|        Kensington|          175|
|           Tribeca|          177|
|          Flushing|          423|
|     East Elmhurst|          183|
|   Upper East Side|         1791|
|          Canarsie|          143|
|    Sheepshead Bay|          163|
|       East Harlem|         1112|
|           Astoria|          898|
|   Lower East Side|          911|
+------------------+-------------+
only showing top 20 rows

None


In [5]:
df.createOrReplaceTempView("airbnb")

avg_price_per_neighborhood = spark.sql("""
    SELECT neighbourhood, AVG(price) AS avg_price
    FROM airbnb
    GROUP BY neighbourhood
""")
print("Average Price per Neighborhood:")
avg_price_per_neighborhood.show()

Average Price per Neighborhood:
+------------------+------------------+
|     neighbourhood|         avg_price|
+------------------+------------------+
|            Corona|         59.171875|
|      Richmondtown|              78.0|
|      Prince's Bay|             409.5|
|       Westerleigh|              71.5|
|        Mill Basin|            179.75|
|      Civic Center|191.94230769230768|
|          40.83166|               1.0|
|        Douglaston| 88.14285714285714|
|        Mount Hope|              77.5|
|       Marble Hill| 89.16666666666667|
|         Rego Park| 83.87735849056604|
|          40.81225|               2.0|
|     Dyker Heights| 93.41666666666667|
| Kew Gardens Hills| 112.3076923076923|
|      Dongan Hills| 79.42857142857143|
|          40.81078|               3.0|
|Financial District|225.49059139784947|
|       Bay Terrace|             142.0|
|          40.83117|               1.0|
|           Midtown| 282.7839065541856|
+------------------+------------------+
only sho

### Calculate the Average Price per Neighborhood

In [6]:
reviews_per_neighborhood = spark.sql("""
    SELECT neighbourhood, COUNT(number_of_reviews) AS total_reviews
    FROM airbnb
    GROUP BY neighbourhood
""")
print("Total Reviews per Neighborhood:")
reviews_per_neighborhood.show()

Total Reviews per Neighborhood:
+------------------+-------------+
|     neighbourhood|total_reviews|
+------------------+-------------+
|            Corona|           64|
|      Richmondtown|            1|
|      Prince's Bay|            4|
|       Westerleigh|            2|
|        Mill Basin|            4|
|      Civic Center|           52|
|          40.83166|            1|
|        Douglaston|            7|
|        Mount Hope|           20|
|       Marble Hill|           12|
|         Rego Park|          106|
|          40.81225|            1|
|     Dyker Heights|           12|
| Kew Gardens Hills|           26|
|      Dongan Hills|            7|
|          40.81078|            1|
|Financial District|          744|
|       Bay Terrace|            6|
|          40.83117|            1|
|           Midtown|         1541|
+------------------+-------------+
only showing top 20 rows



### Calculate the Average Price and Total Reviews per Neighborhood:

In [9]:
avg_price_and_reviews = spark.sql("""
    SELECT neighbourhood, AVG(price) AS avg_price, COUNT(number_of_reviews) AS total_reviews
    FROM airbnb
    GROUP BY neighbourhood
""")
print("Average Price and Total Reviews per Neighborhood:")
avg_price_and_reviews.show()

Average Price and Total Reviews per Neighborhood:
+------------------+------------------+-------------+
|     neighbourhood|         avg_price|total_reviews|
+------------------+------------------+-------------+
|            Corona|         59.171875|           64|
|      Richmondtown|              78.0|            1|
|      Prince's Bay|             409.5|            4|
|       Westerleigh|              71.5|            2|
|        Mill Basin|            179.75|            4|
|      Civic Center|191.94230769230768|           52|
|          40.83166|               1.0|            1|
|        Douglaston| 88.14285714285714|            7|
|        Mount Hope|              77.5|           20|
|       Marble Hill| 89.16666666666667|           12|
|         Rego Park| 83.87735849056604|          106|
|          40.81225|               2.0|            1|
|     Dyker Heights| 93.41666666666667|           12|
| Kew Gardens Hills| 112.3076923076923|           26|
|      Dongan Hills| 79.42857142

## Amazon Sale Data

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("AmazonDataProcessing") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://hdfs-namenode:9000") \
    .getOrCreate()

# Read the Amazon data from HDFS
amazon_data = "hdfs://hdfs-namenode:9000/datasets" + "/Amazon Sale Report.csv"
amazon_data_df = spark.read.csv(amazon_data, header=True, inferSchema=True)


total_sales_per_city = amazon_data_df.groupBy("ship-city").agg(sum("Amount").alias("total_sales"))

total_sales_per_city.show()
# print(amazon_data_df.show())


+--------------------+------------------+
|           ship-city|       total_sales|
+--------------------+------------------+
|             GWALIOR|          91540.91|
|           Bangalore|441524.61999999994|
|   Avinashi, Tirupur|            9507.0|
|NANDIVARAM GUDUVA...|29664.420000000002|
|         KODUNGALLUR|            4991.0|
|            GULAOTHI|             476.0|
|            Zirakpur| 8929.619999999999|
|        Kudukkimotta|            3766.0|
|        CHEKONIDHARA|            2225.0|
|            JHINJHAK|            1288.0|
|            DHAULPUR|            4951.0|
|               JOWAI|           6179.29|
|        YELAMANCHILI|            3217.0|
|         SIKANDRABAD|            1186.0|
|          KUMARAGIRI|             517.0|
|              CHOPRA|             547.0|
|            TALIKOTA|            1532.0|
|             Udaipur|          17404.15|
|ATMAKUR SRI POTTI...|            4739.0|
|         JillelaGuda|            1740.0|
+--------------------+------------

In [18]:
amazon_data_df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: double (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Unnamed: 22: boolean (nullable = true)



In [25]:
amazon_data_df.show()

+-----+-------------------+--------+--------------------+----------+--------------+------------------+--------+-------------------+-------------+----+----------+--------------+---+--------+------+-----------+--------------+----------------+------------+--------------------+-----+------------+-----------+
|index|           Order ID|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|   Style|                SKU|     Category|Size|      ASIN|Courier Status|Qty|currency|Amount|  ship-city|    ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|
+-----+-------------------+--------+--------------------+----------+--------------+------------------+--------+-------------------+-------------+----+----------+--------------+---+--------+------+-----------+--------------+----------------+------------+--------------------+-----+------------+-----------+
|    0|405-8078784-5731545|04-30-22|           Cancelled|  Merchant|     Amazon.in